# 集成算法
> 集成算法一般包括有：`Bagging, Boosting, Voting`三种；

- **Bagging 算法：**先将训练集分离成多个子集，然后通过各个子集训练多个模型；
- **Boosting 算法：**训练多个模型并组成一个序列，序列中的每一个模型都会修正前一个模型的错误；
- **Voting 算法：**训练多个模型，并采用样本统计来提高模型的准确度；

## `Bagging`(装袋算法)
> 装袋算法是一种提高分类准确率的算法，通过给定组合投票的方式获得最优解。比如你生病了，去n个医院看了n个医生，每个医生都给你开了药方，最后哪个药方的出现次数多，就说明这个药方越有可能是最优解，这很好理解，这也是装袋算法的思想;

### 1. `Bagged Decision Tree`
> **装袋算法在数据具有很大的方差时非常有效**，最常见的例子就是决策树的装袋算法。

In [ ]:
# 导入包
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# 导入数据
filename = './data/data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(filename, names=names)

# 将数据分为输入数据和输出结果
array = data.values
X = array[:, 0:8]
y = array[:, 8]

# 把训练数据集分成10份子集
num_folds = 10
seed = 7
kfold = KFold(n_splits=num_folds, random_state=seed)
cart = DecisionTreeClassifier()
num_tree = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_tree, random_state=seed)
result = cross_val_score(model, X, y, cv=kfold)
print(result.mean())

### 2. `Random Forest`

所谓的随机森林就是指用随机的方式构建一个森林，森林由决策树组成，每个决策树之间是相互独立的，得到森林之后，当有一个新的输入样本进入的时候，就让森林中的每一棵决策树分别进行判断，看看这个样本应该属于哪一类，再看看哪一类被选择最多，就预测这个样本为哪一类。

> NOTE:在建立每一棵决策树的过程中，有两点需要注意：**采样**与**完全分裂**。

+ 采样：
    随机森林在构建决策树时有两个随机采样的过程：对输入的数据进行*行采样*和*列采样*；
    
    对于*行采样*是采用**有放回**的方式，即在采样的结果当中可能存在有重复的样本。假设输入样本为N个，那么采样的样本也为N个，这样在训练过程中每一棵决策树的输入样本都不是全部样本，这样能一定程度上**避免过拟合**；
    
    对于*列采样*，从M个特征中选出m(m<<M)个特征，之后再对采样之后的数据使用**完全分裂**的方式构建决策树，这样决策树的某一个叶子节点要么是无法继续分裂的，要么所有样本都指向同一个分类。一般很多的决策树算法都有一个重要的步骤——剪枝，但是这里不这么做，因为之前的两个随机采样过程保证了随机性，所以不剪枝也不会出现过拟合。
    
#### Reference：

[random_forest wiki](https://en.wikipedia.org/wiki/Random_forest)

[随机森林详解](http://blog.csdn.net/y0367/article/details/51501780)


In [ ]:
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
# 导入数据
filename = './data/data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(filename, names=names)
# 将数据分为输入数据和输出结果
array = data.values
X = array[:, 0:8]
Y = array[:, 8]
num_folds = 10
seed = 7
kfold = KFold(n_splits=num_folds, random_state=seed)

# 生成100棵树
num_tree = 100
max_features = 3
model = RandomForestClassifier(n_estimators=num_tree, random_state=seed, max_features=max_features)
result = cross_val_score(model, X, Y, cv=kfold)
print(result.mean())

## `Boosting`(提升算法)
> 提升算法是一种用来提高弱分类算法准确度的方法，这种方法**先构造一个预测函数系列，然后以一定的方式将它们组合成一个预测函数**。提升算法也是一种提高任意给定学习算法准确度的方法，它是一种集成算法，主要通过对样本集的操作获得样本子集，然后用弱分类算法在样本子集上训练生成一系列的基分类器。它可以用来提高其他弱分类算法的识别率，也就是将其他的弱分类算法作为基分类算法放于提升框架中，通过提升框架对训练样本集的操作，得到不同的训练样本子集，再用该样本子集去训练生成基分类器。每得到一个样本集就用该基分类算法在该样本集上产生一个基分类器，这样在给定训练轮数n后，就可产生n个基分类器，然后提升算法将这n个基分类器进行加权融合，产生最后的结果分类器。在这n个基分类器中，每个分类器的识别率不一定很高，但它们联合后的结果有很高的识别率，这样便提高了弱分类算法的识别率。

### 1. AdaBoost
> AdaBoost是一种迭代算法，其核心思想是针对同一个训练集训练不同的分类器（弱分类器），然后把这些弱分类器集合起来，构成一个更强的最终分类器（强分类器）。其算法本身是通过改变数据分布来实现的，**它根据每次训练集中每个样本的分类是否正确，以及上次的总体分类的准确率，来确定每个样本的权值**。它将*修改过权值的新数据集送给下层分类器进行训练，再将每次训练得到的分类器融合起来，作为最后的决策分类器*。使用AdaBoost分类器可以排除一些不必要的训练数据特征，并放在关键的训练数据上面；

#### Reference
[Adaboost详解](http://blog.csdn.net/llp1992/article/details/46514165)

In [ ]:
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifie
# 导入数据
filename = './data/data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(filename, names=names)
# 将数据分为输入数据和输出结果
array = data.values
X = array[:, 0:8]
Y = array[:, 8]

num_folds = 10
seed = 7
kfold = KFold(n_splits=num_folds, random_state=seed)
num_tree = 30
model = AdaBoostClassifier(n_estimators=num_tree, random_state=seed)
result = cross_val_score(model, X, Y, cv=kfold)
print(result.mean())

### 2. Gradient Tree Boosting(随机梯度提升算法)
> Gradient Tree Boosting(别名 GBM, GBRT, GBDT, MART)，随机梯度提升法（GBM）基于的思想是：要找到某个函数的最大值，最好的办法就是沿着该函数的梯度方向探寻。梯度算子总是指向函数值增长最快的方向。由于梯度提升算法在每次更新数据集时都需要遍历整个数据集，计算复杂度较高，于是有了一个改进算法——随机梯度提升算法，该算法一次只用一个样本点来更新回归系数，极大地改善了算法的计算复杂度。

#### Reference
[从GBDT-Xgboost](http://blog.csdn.net/china1000/article/details/51106856)
[一步一步理解GB、GBDT、xgboost](http://blog.csdn.net/shenxiaoming77/article/details/51542982)
[Gradient Tree Boosting算法解析]

In [ ]:
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
# 导入数据
filename = './data/data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(filename, names=names)
# 将数据分为输入数据和输出结果
array = data.values
X = array[:, 0:8]
Y = array[:, 8]

num_folds = 10
seed = 7
kfold = KFold(n_splits=num_folds, random_state=seed)
num_tree = 100
model = GradientBoostingClassifier(n_estimators=num_tree, random_state=seed)
result = cross_val_score(model, X, Y, cv=kfold)
print(result.mean())

## `Voting`
> 投票算法（Voting）是一个非常简单的多个机器学习算法的集成算法。投票算法是通过创建两个或多个算法模型，利用投票算法将这些算法包装起来，计算各个子模型的平均预测状况。在实际的应用中，可以对每个子模型的预测结果增加权重，以提高算法的准确度。但是，在scikit-learn中不提供加权算法。下面通过一个例子来展示在scikit-learn中如何实现一个投票算法。

In [ ]:
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
# 导入数据
filename = './data/data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(filename, names=names)
# 将数据分为输入数据和输出结果
array = data.values
X = array[:, 0:8]
Y = array[:, 8]

num_folds = 10
seed = 7
kfold = KFold(n_splits=num_folds, random_state=seed)
cart = DecisionTreeClassifier()
models = []
model_logistic = LogisticRegression()
models.append(('logistic', model_logistic))
model_cart = DecisionTreeClassifier()
models.append(('cart', model_cart))
model_svc = SVC()
models.append(('svm', model_svc))
ensemble_model = VotingClassifier(estimators=models)
result = cross_val_score(ensemble_model, X, Y, cv=kfold)
print(result.mean())